In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os


cat_directory = './data/training/catalogs/'
cat_file = cat_directory + 'S12_Grade_A.csv'
cat = pd.read_csv(cat_file)

row = cat.iloc[23]
arrival_time = datetime.strptime(row['time_abs(%Y-%m-%dT%H:%M:%S.%f)'],'%Y-%m-%dT%H:%M:%S.%f')
arrival_time

In [ ]:
# Load data
seismic_data = pd.read_csv('seismic_data.csv')
event_data = arrival_time

# Convert time_abs to datetime
seismic_data['time_abs'] = pd.to_datetime(seismic_data['time_abs'])
event_data['time_abs'] = pd.to_datetime(event_data['time_abs'])

In [ ]:
# Function to extract data around event times
def get_event_window(seismic_df, event_time, window_size):
    start_time = event_time - pd.Timedelta(seconds=window_size)
    end_time = event_time + pd.Timedelta(seconds=window_size)
    return seismic_df[(seismic_df['time_abs'] >= start_time) & (seismic_df['time_abs'] <= end_time)]


In [ ]:
from scipy.signal import butter, filtfilt

# Define filter functions
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs  # Nyquist Frequency
    low = lowcut / nyq
    high = highcut / nyq
    return butter(order, [low, high], btype='band')

def apply_filter(data, lowcut, highcut, fs):
    b, a = butter_bandpass(lowcut, highcut, fs)
    y = filtfilt(b, a, data)
    return y

# Apply filter to seismic velocity data
fs = 1 / seismic_data['time_rel'].diff().median()  # Sampling frequency
seismic_data['velocity_filtered'] = apply_filter(seismic_data['velocity'], lowcut=0.1, highcut=10.0, fs=fs)


In [ ]:
from scipy.signal import detrend

seismic_data['velocity_detrended'] = detrend(seismic_data['velocity_filtered'])



from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
seismic_data['velocity_normalized'] = scaler.fit_transform(seismic_data[['velocity_detrended']])



seismic_data['velocity_normalized'].interpolate(method='linear', inplace=True)


import numpy as np

z_scores = np.abs((seismic_data['velocity_normalized'] - seismic_data['velocity_normalized'].mean()) / seismic_data['velocity_normalized'].std())
seismic_data = seismic_data[z_scores < 3]




In [ ]:
peak_amplitude = seismic_data['velocity_normalized'].max()


mean_velocity = seismic_data['velocity_normalized'].mean()
std_velocity = seismic_data['velocity_normalized'].std()


zero_crossings = ((seismic_data['velocity_normalized'][:-1] * seismic_data['velocity_normalized'][1:]) < 0).sum()


autocorr = seismic_data['velocity_normalized'].autocorr(lag=1)


fft_values = np.fft.fft(seismic_data['velocity_normalized'])
fft_freq = np.fft.fftfreq(len(fft_values), d=1/fs)

dominant_freq = fft_freq[np.argmax(np.abs(fft_values))]

spectral_centroid = np.sum(fft_freq * np.abs(fft_values)) / np.sum(np.abs(fft_values))


In [ ]:
def band_energy(fft_vals, freq_bins, low_freq, high_freq):
    indices = np.where((freq_bins >= low_freq) & (freq_bins <= high_freq))
    return np.sum(np.abs(fft_vals[indices]) ** 2)

total_energy = np.sum(np.abs(fft_values) ** 2)
low_band_energy = band_energy(fft_values, fft_freq, 0.1, 1.0)
high_band_energy = band_energy(fft_values, fft_freq, 1.0, 10.0)
band_energy_ratio = low_band_energy / total_energy


import pywt

scales = np.arange(1, 128)
coefficients, frequencies = pywt.cwt(seismic_data['velocity_normalized'], scales, 'morl')



wavelet_features = coefficients.flatten()


In [ ]:
window_size = 30  # seconds
event_windows = []

for _, event in event_data.iterrows():
    event_time = event['time_abs']
    window_data = get_event_window(seismic_data, event_time, window_size)
    event_windows.append(window_data)


seismic_data['label'] = 0  # Default to No Event

for window in event_windows:
    seismic_data.loc[window.index, 'label'] = 1

class_counts = seismic_data['label'].value_counts()
print(class_counts)


In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_resample(seismic_data.drop('label', axis=1), seismic_data['label'])


from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(seismic_data.drop('label', axis=1), seismic_data['label'])


In [ ]:
window_length = 128  # Number of samples per window
step_size = 64       # Overlap between windows

def create_windows(data, window_length, step_size):
    windows = []
    labels = []
    for i in range(0, len(data) - window_length, step_size):
        window = data.iloc[i:i + window_length]
        windows.append(window.drop('label', axis=1).values)
        labels.append(window['label'].mode()[0])  # Majority label in the window
    return np.array(windows), np.array(labels)

X, y = create_windows(seismic_data, window_length, step_size)


In [ ]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp)


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    # Train and validate your model here


In [ ]:
# Example of Wiener filter
from scipy.signal import wiener

seismic_data['velocity_adaptive_filtered'] = wiener(seismic_data['velocity_normalized'])


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train.reshape(len(X_train), -1), y_train)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(window_length, num_features)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)


In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

f, t, Sxx = spectrogram(seismic_data['velocity_normalized'], fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()


In [ ]:
plt.plot(seismic_data['time_abs'], seismic_data['velocity_normalized'])
plt.xlabel('Time')
plt.ylabel('Normalized Velocity')
plt.title('Seismic Wiggle Plot')
plt.show()


In [ ]:
noise = np.random.normal(0, 0.01, seismic_data['velocity_normalized'].shape)
seismic_data['velocity_augmented'] = seismic_data['velocity_normalized'] + noise

seismic_data['velocity_shifted'] = seismic_data['velocity_normalized'].shift(1).fillna(0)


In [ ]:
median_velocity = seismic_data['velocity_normalized'].median()
mad_velocity = seismic_data['velocity_normalized'].mad()


In [ ]:
from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(contamination=0.1)
anomalies = iso_forest.fit_predict(seismic_data[['velocity_normalized']])
seismic_data = seismic_data[anomalies == 1]
